In [1]:
import os
import glob
import numpy as np
import yaml
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO

/home/cbddobvyz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cocoModelName = 'yolo11m.pt'
ketemModelName = 'ketemYoloMedium.pt'
dataYamlPath = 'data.yaml'
projectFolder = 'objectDetection'
trainingFolder = 'inbreastTraining'
validationFolder = 'inbreastValidation'

# DATA INFORMATION

In [3]:
with open(dataYamlPath) as stream:
    dataYamlDict = yaml.safe_load(stream)

classes = dataYamlDict['names']
class_size = len(classes)

print('Class names:', classes)

Class names: ['MASS']


In [4]:
splitPath = dataYamlDict['train'].split(os.path.sep)
splitPath[-1] = 'labels'

trainLabelFolderPath = os.path.sep.join(splitPath)
trainLabelFilePaths = glob.glob(os.path.join(trainLabelFolderPath, '*'))

trainImageCounts = [0] * class_size
totalLabelCounts = np.array([0] * class_size, dtype=int)

for k in range(len(trainLabelFilePaths)):
    
    trainLabelFilePath = trainLabelFilePaths[k]
    
    fileContents = open(trainLabelFilePath, 'r')
    labelCounts = np.array([0] * class_size)
    for fileContent in fileContents:
        data = fileContent.strip()
        labelCounts[int(data[0])]+=1    
    
    totalLabelCounts += labelCounts    
    
    for j in range(len(labelCounts)):
        if labelCounts[j]>0:
            trainImageCounts[j]+=1
            
for i in range(len(trainImageCounts)):
    print(classes[i], 'class counts in', trainImageCounts[i], 'images in TRAINING with', totalLabelCounts[i], 'labels.')

MASS class counts in 87 images in TRAINING with 96 labels.


In [5]:
splitPath = dataYamlDict['val'].split(os.path.sep)
splitPath[-1] = 'labels'

testLabelFolderPath = os.path.sep.join(splitPath)
testLabelFilePaths = glob.glob(os.path.join(testLabelFolderPath, '*'))

testImageCounts = [0] * class_size
totalLabelCounts = np.array([0] * class_size, dtype=int)

for k in range(len(testLabelFilePaths)):
    
    testLabelFilePath = testLabelFilePaths[k]
    
    fileContents = open(testLabelFilePath, 'r')
    labelCounts = np.array([0] * class_size)
    for fileContent in fileContents:
        data = fileContent.strip()
        labelCounts[int(data[0])]+=1    
    
    totalLabelCounts += labelCounts    
    
    for j in range(len(labelCounts)):
        if labelCounts[j]>0:
            testImageCounts[j]+=1
            
for i in range(len(testImageCounts)):
    print(classes[i], 'class counts in', testImageCounts[i], 'images in TEST with', totalLabelCounts[i], 'labels.')

MASS class counts in 20 images in TEST with 20 labels.


# Loading COCO PRETRAINED MODEL

In [6]:
cocoModel = YOLO(cocoModelName)

## MODEL TRAINING

In [7]:
# Train the cocoModel
cocoTrainResults = cocoModel.train(
    data=dataYamlPath,  # path to dataset YAML
    epochs=30,  # number of training epochs
    imgsz=640,  # training image size
    device=[0,1],  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    project=projectFolder,
    name=trainingFolder + 'withCOCO'
)

New https://pypi.org/project/ultralytics/8.3.55 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
                                                        CUDA:1 (Tesla V100-SXM2-32GB, 32510MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=objectDetection, name=inbreastTrainingwithCOCO, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=Fa

train: Scanning /workspace/notebooks/webinar/data/Fold0/train/labels.cache... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<?, ?it/s]
val: Scanning /workspace/notebooks/webinar/data/Fold0/validation/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to objectDetection/inbreastTrainingwithCOCO/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 16 dataloader workers
Logging results to objectDetection/inbreastTrainingwithCOCO
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      4.55G      2.522      5.799      2.037          8        640: 100%|██████████| 6/6 [00:04<00:00,  1.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.81it/s]


                   all         20         20   0.000833       0.25     0.0019    0.00162

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         20         20       0.85        0.9      0.862      0.553

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      4.75G     0.8307     0.6411     0.9816          4        640: 100%|██████████| 6/6 [00:01<00:00,  5.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 14.99it/s]


                   all         20         20      0.806        0.9      0.848      0.529

30 epochs completed in 0.022 hours.
Optimizer stripped from objectDetection/inbreastTrainingwithCOCO/weights/last.pt, 40.5MB
Optimizer stripped from objectDetection/inbreastTrainingwithCOCO/weights/best.pt, 40.5MB

Validating objectDetection/inbreastTrainingwithCOCO/weights/best.pt...
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
                                                        CUDA:1 (Tesla V100-SXM2-32GB, 32510MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 10.30it/s]


                   all         20         20      0.851        0.9      0.862      0.554
Speed: 1.8ms preprocess, 3.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to objectDetection/inbreastTrainingwithCOCO


## Metric Calculations

### Evaluation Metrics

In [8]:
cocoValidationResult = cocoModel.val(project=projectFolder, name=validationFolder + 'withCOCO', save_json=True, data=dataYamlPath)

Ultralytics 8.3.51 🚀 Python-3.10.12 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
                                                        CUDA:1 (Tesla V100-SXM2-32GB, 32510MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning /workspace/notebooks/webinar/data/Fold0/validation/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


                   all         20         20      0.851        0.9      0.864      0.555
Speed: 4.0ms preprocess, 6.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Saving objectDetection/inbreastValidationwithCOCO/predictions.json...
Results saved to objectDetection/inbreastValidationwithCOCO


#### Recall - Precision - AP - mAP Precision

In [9]:
for i, className in enumerate(classes):
    print(className, 'Recall Value:', cocoValidationResult.box.r.tolist()[i])
    print(className, 'Precision Value:', cocoValidationResult.box.p.tolist()[i])
    print(className, 'AP Value:', cocoValidationResult.box.ap50.tolist()[i])
    print(className, 'mAP Value:', cocoValidationResult.box.map50.tolist())

MASS Recall Value: 0.9
MASS Precision Value: 0.8505999539642123
MASS AP Value: 0.8641147706222079
MASS mAP Value: 0.8641147706222079


#### Confusion Matrix

In [10]:
plt.imshow(cv2.imread(os.path.join(cocoValidationResult.save_dir, 'confusion_matrix.png')))

In [11]:
cocoTruePositiveCount, cocoFalsePositiveCount = cocoValidationResult.confusion_matrix.tp_fp()
cocoFalseNegativeCount = totalLabelCounts - cocoTruePositiveCount
print('True Positive Count:', cocoTruePositiveCount, 'False Positive Count:', cocoFalsePositiveCount, 'False Negative Count:', cocoFalseNegativeCount)

True Positive Count: [         18] False Positive Count: [          3] False Negative Count: [          2]


### Speed Metrics

In [12]:
print('Preprocess Time:', cocoValidationResult.speed['preprocess'])
print('Inference Time:', cocoValidationResult.speed['inference'])
print('Postprocess Time:', cocoValidationResult.speed['postprocess'])

Preprocess Time: 3.9660215377807617
Inference Time: 6.361150741577148
Postprocess Time: 0.5598545074462891


## PREDICTIONS ON TEST DATA AND VISUALIZING MODEL RESULTS

In [13]:
cocoModelFilePath = os.path.join(projectFolder, trainingFolder + 'withCOCO', 'weights', 'best.pt')
cocoModel = YOLO(cocoModelFilePath)
cocoModelResults = cocoModel.predict(project=projectFolder, name=validationFolder+'PredswithCOCO', source=glob.glob(os.path.join(dataYamlDict['val'], '*')), save=True)


0: 640x640 1 MASS, 7.4ms
1: 640x640 1 MASS, 7.4ms
2: 640x640 (no detections), 7.4ms
3: 640x640 2 MASSs, 7.4ms
4: 640x640 1 MASS, 7.4ms
5: 640x640 1 MASS, 7.4ms
6: 640x640 1 MASS, 7.4ms
7: 640x640 (no detections), 7.4ms
8: 640x640 1 MASS, 7.4ms
9: 640x640 2 MASSs, 7.4ms
10: 640x640 1 MASS, 7.4ms
11: 640x640 1 MASS, 7.4ms
12: 640x640 1 MASS, 7.4ms
13: 640x640 1 MASS, 7.4ms
14: 640x640 3 MASSs, 7.4ms
15: 640x640 1 MASS, 7.4ms
16: 640x640 1 MASS, 7.4ms
17: 640x640 1 MASS, 7.4ms
18: 640x640 1 MASS, 7.4ms
19: 640x640 2 MASSs, 7.4ms
Speed: 4.1ms preprocess, 7.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to objectDetection/inbreastValidationPredswithCOCO


In [14]:
cocoModelResults[0].summary()

[{'name': 'MASS',
  'class': 0,
  'confidence': 0.72237,
  'box': {'x1': 3038.61548, 'y1': 1227.81592, 'x2': 3328.0, 'y2': 1410.18811}}]

In [15]:
cocoModelResults[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.7224], device='cuda:0')
data: tensor([[3.0386e+03, 1.2278e+03, 3.3280e+03, 1.4102e+03, 7.2237e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (4084, 3328)
shape: torch.Size([1, 6])
xywh: tensor([[3183.3076, 1319.0020,  289.3845,  182.3722]], device='cuda:0')
xywhn: tensor([[0.9565, 0.3230, 0.0870, 0.0447]], device='cuda:0')
xyxy: tensor([[3038.6155, 1227.8159, 3328.0000, 1410.1881]], device='cuda:0')
xyxyn: tensor([[0.9130, 0.3006, 1.0000, 0.3453]], device='cuda:0')

In [16]:
fileIndex = 1

fileName = cocoModelResults[fileIndex].path.split(os.path.sep)[-1].split('.')[0]
predImagePath = os.path.join(os.getcwd(), projectFolder, validationFolder+'PredswithCOCO', fileName + '.jpg')
predImage = cv2.imread(predImagePath)

plt.figure(figsize=(5,5))
plt.imshow(predImage)

# Loading KETEM PRETRAINED MODEL

In [17]:
ketemModel = YOLO(ketemModelName)

## MODEL TRAINING

In [18]:
# Train the cocoModel
train_results = ketemModel.train(
    data=dataYamlPath,  # path to dataset YAML
    epochs=30,  # number of training epochs
    imgsz=640,  # training image size
    device=[0,1],  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    project=projectFolder,
    name=trainingFolder + 'withKETEM'
)

New https://pypi.org/project/ultralytics/8.3.55 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
                                                        CUDA:1 (Tesla V100-SXM2-32GB, 32510MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ketemYoloMedium.pt, data=data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=objectDetection, name=inbreastTrainingwithKETEM, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=Fals

train: Scanning /workspace/notebooks/webinar/data/Fold0/train/labels.cache... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<?, ?it/s]
val: Scanning /workspace/notebooks/webinar/data/Fold0/validation/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to objectDetection/inbreastTrainingwithKETEM/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 16 dataloader workers
Logging results to objectDetection/inbreastTrainingwithKETEM
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      4.55G      1.429      4.469      1.228          8        640: 100%|██████████| 6/6 [00:04<00:00,  1.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.59it/s]


                   all         20         20      0.001        0.3    0.00717    0.00476

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         20         20      0.889       0.85        0.9      0.615

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      4.75G     0.8532     0.7857     0.9391          4        640: 100%|██████████| 6/6 [00:01<00:00,  5.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 15.42it/s]


                   all         20         20      0.888       0.85      0.892       0.61

30 epochs completed in 0.022 hours.
Optimizer stripped from objectDetection/inbreastTrainingwithKETEM/weights/last.pt, 40.5MB
Optimizer stripped from objectDetection/inbreastTrainingwithKETEM/weights/best.pt, 40.5MB

Validating objectDetection/inbreastTrainingwithKETEM/weights/best.pt...
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
                                                        CUDA:1 (Tesla V100-SXM2-32GB, 32510MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  9.23it/s]


                   all         20         20       0.95      0.947      0.975       0.66
Speed: 1.7ms preprocess, 3.3ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to objectDetection/inbreastTrainingwithKETEM


## Metric Calculations

### Evaluation Metrics

In [19]:
ketemValidationResult = ketemModel.val(project=projectFolder, name=validationFolder + 'withKETEM', save_json=True, data=dataYamlPath)

Ultralytics 8.3.51 🚀 Python-3.10.12 torch-1.12.1+cu116 CUDA:0 (Tesla V100-SXM2-32GB, 32510MiB)
                                                        CUDA:1 (Tesla V100-SXM2-32GB, 32510MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning /workspace/notebooks/webinar/data/Fold0/validation/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.64s/it]


                   all         20         20       0.95      0.944      0.978      0.664
Speed: 3.0ms preprocess, 6.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Saving objectDetection/inbreastValidationwithKETEM/predictions.json...
Results saved to objectDetection/inbreastValidationwithKETEM


#### Recall - Precision - AP - mAP Precision

In [20]:
for className in classes:
    print(className, 'Recall Value:', ketemValidationResult.box.r.tolist())
    print(className, 'Precision Value:', ketemValidationResult.box.p.tolist())
    print(className, 'AP Value:', ketemValidationResult.box.ap50.tolist())
    print(className, 'mAP Value:', ketemValidationResult.box.map50.tolist())

MASS Recall Value: [0.9436361208852634]
MASS Precision Value: [0.9496650589939611]
MASS AP Value: [0.9775]
MASS mAP Value: 0.9775


#### Confusion Matrix

In [21]:
plt.imshow(cv2.imread(os.path.join(ketemValidationResult.save_dir, 'confusion_matrix.png')))

In [22]:
ketemTruePositiveCount, ketemFalsePositiveCount = ketemValidationResult.confusion_matrix.tp_fp()
ketemFalseNegativeCount = totalLabelCounts - ketemTruePositiveCount
print('True Positive Count:', ketemTruePositiveCount, 'False Positive Count:', ketemFalsePositiveCount, 'False Negative Count:', ketemFalseNegativeCount)

True Positive Count: [         19] False Positive Count: [          1] False Negative Count: [          1]


### Speed Metrics

In [23]:
print('Preprocess Time:', ketemValidationResult.speed['preprocess'])
print('Inference Time:', ketemValidationResult.speed['inference'])
print('Postprocess Time:', ketemValidationResult.speed['postprocess'])

Preprocess Time: 3.0417323112487793
Inference Time: 6.824588775634766
Postprocess Time: 2.0153045654296875


## PREDICTIONS ON TEST DATA AND VISUALIZING MODEL RESULTS

In [24]:
ketemModelFilePath = os.path.join(projectFolder, trainingFolder + 'withKETEM', 'weights', 'best.pt')
ketemModel = YOLO(ketemModelFilePath)
ketemModelResults = ketemModel.predict(project=projectFolder, name=validationFolder+'PredswithKETEM', source=glob.glob(os.path.join(dataYamlDict['val'], '*')), save=True)


0: 640x640 1 MASS, 7.0ms
1: 640x640 1 MASS, 7.0ms
2: 640x640 (no detections), 7.0ms
3: 640x640 1 MASS, 7.0ms
4: 640x640 1 MASS, 7.0ms
5: 640x640 1 MASS, 7.0ms
6: 640x640 1 MASS, 7.0ms
7: 640x640 1 MASS, 7.0ms
8: 640x640 1 MASS, 7.0ms
9: 640x640 1 MASS, 7.0ms
10: 640x640 1 MASS, 7.0ms
11: 640x640 1 MASS, 7.0ms
12: 640x640 1 MASS, 7.0ms
13: 640x640 1 MASS, 7.0ms
14: 640x640 1 MASS, 7.0ms
15: 640x640 1 MASS, 7.0ms
16: 640x640 1 MASS, 7.0ms
17: 640x640 1 MASS, 7.0ms
18: 640x640 1 MASS, 7.0ms
19: 640x640 1 MASS, 7.0ms
Speed: 6.7ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to objectDetection/inbreastValidationPredswithKETEM


In [25]:
ketemModelResults[0].summary()

[{'name': 'MASS',
  'class': 0,
  'confidence': 0.73649,
  'box': {'x1': 3040.53735, 'y1': 1218.45471, 'x2': 3328.0, 'y2': 1414.08801}}]

In [26]:
ketemModelResults[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.7365], device='cuda:0')
data: tensor([[3.0405e+03, 1.2185e+03, 3.3280e+03, 1.4141e+03, 7.3649e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (4084, 3328)
shape: torch.Size([1, 6])
xywh: tensor([[3184.2686, 1316.2714,  287.4626,  195.6333]], device='cuda:0')
xywhn: tensor([[0.9568, 0.3223, 0.0864, 0.0479]], device='cuda:0')
xyxy: tensor([[3040.5374, 1218.4547, 3328.0000, 1414.0880]], device='cuda:0')
xyxyn: tensor([[0.9136, 0.2983, 1.0000, 0.3463]], device='cuda:0')

## Benchmarking 

In [27]:
for i, className in enumerate(classes):
    print(className, 'Recall Value in KETEM Pretrained:', round(ketemValidationResult.box.r.tolist()[i],3), 'Recall Value in COCO Pretrained:', round(cocoValidationResult.box.r.tolist()[i],3))
    print(className, 'Precision Value in KETEM Pretrained:', round(ketemValidationResult.box.p.tolist()[i],3), 'Precision Value in COCO Pretrained:', round(cocoValidationResult.box.p.tolist()[i],3))
    print(className, 'AP Value in KETEM Pretrained:', round(ketemValidationResult.box.ap50.tolist()[i],3), 'AP Value in COCO Pretrained:', round(cocoValidationResult.box.ap50.tolist()[i],3))
    print(className, 'mAP Value in KETEM Pretrained:', round(ketemValidationResult.box.map50.tolist(),3), 'mAP Value in COCO Pretrained:', round(cocoValidationResult.box.map50.tolist(),3))

MASS Recall Value in KETEM Pretrained: 0.944 Recall Value in COCO Pretrained: 0.9
MASS Precision Value in KETEM Pretrained: 0.95 Precision Value in COCO Pretrained: 0.851
MASS AP Value in KETEM Pretrained: 0.978 AP Value in COCO Pretrained: 0.864
MASS mAP Value in KETEM Pretrained: 0.978 mAP Value in COCO Pretrained: 0.864


In [28]:
dfData = {
    'KETEM Recall': ketemValidationResult.box.r.tolist(),
    'COCO Recall': cocoValidationResult.box.r.tolist(),
    'KETEM Precision': ketemValidationResult.box.p.tolist(),
    'COCO Precision': cocoValidationResult.box.p.tolist(),
    'KETEM FPPI': ketemFalsePositiveCount/np.array(testImageCounts),
    'COCO FPPI': cocoFalsePositiveCount/np.array(testImageCounts),
    'KETEM AP': ketemValidationResult.box.ap50.tolist(),
    'COCO AP': cocoValidationResult.box.ap50.tolist(),
    'KETEM mAP': [ketemValidationResult.box.map50.tolist()],
    'COCO mAP': [cocoValidationResult.box.map50.tolist()],
    'KETEM TP': ketemTruePositiveCount,
    'COCO TP': cocoTruePositiveCount,
    'KETEM FP': ketemFalsePositiveCount,
    'COCO FP': cocoFalsePositiveCount,
    'KETEM FN': ketemFalseNegativeCount,
    'COCO FN': cocoFalseNegativeCount,
}
benchmarkDf = pd.DataFrame(data=dfData, index=classes)
benchmarkDf

,KETEM Recall,COCO Recall,KETEM Precision,COCO Precision,KETEM FPPI,COCO FPPI,KETEM AP,COCO AP,KETEM mAP,COCO mAP,KETEM TP,COCO TP,KETEM FP,COCO FP,KETEM FN,COCO FN
MASS,0.943636,0.9,0.949665,0.8506,0.05,0.15,0.9775,0.864115,0.9775,0.864115,19.0,18.0,1.0,3.0,1.0,2.0


In [29]:
ketemValResultList = [ketemValidationResult.box.r.tolist()[0],
                     ketemValidationResult.box.p.tolist()[0],
                     (ketemFalsePositiveCount/np.array(testImageCounts))[0],
                     ketemValidationResult.box.ap50.tolist()[0],
                     ketemValidationResult.box.map50.tolist(),
                     ketemTruePositiveCount[0],
                     ketemFalsePositiveCount[0],
                     ketemFalseNegativeCount[0]
                    ]
cocoValResultList = [cocoValidationResult.box.r.tolist()[0],
                     cocoValidationResult.box.p.tolist()[0],
                     (cocoFalsePositiveCount/np.array(testImageCounts))[0],
                     cocoValidationResult.box.ap50.tolist()[0],
                     cocoValidationResult.box.map50.tolist(),
                     cocoTruePositiveCount[0],
                     cocoFalsePositiveCount[0],
                     cocoFalseNegativeCount[0]
                    ]
benchmarkDf = pd.DataFrame(data=[ketemValResultList, cocoValResultList], index=['KETEM', 'COCO'], columns=['Recall', 'Precision', 'FPPI', 'AP', 'mAP', 'TP', 'FP', 'FN'])
benchmarkDf

,Recall,Precision,FPPI,AP,mAP,TP,FP,FN
KETEM,0.943636,0.949665,0.05,0.977500,0.977500,19.0,1.0,1.0
COCO,0.900000,0.850600,0.15,0.864115,0.864115,18.0,3.0,2.0
